# Imports

In [2]:
import numpy as np
import pandas as pd
from itertools import product
from numba import njit
import time
%matplotlib inline

# Preparing Data

In [3]:
pauwel = pd.read_csv("pauwel-dataset.txt", sep='\t', header=0)
pauwel

,abdominal cramps,abdominal distention,abdominal pain,malformations,spontaneous abortion,missed abortion,abscess,acanthosis nigricans,acidosis,renal tubular acidosis,...,drug dependence,diverticulosis,prostatic hypertrophy,allergic reaction,dysphonia,eosinophilic pneumonia,retinal vein thrombosis,renal insufficiency,glioblastoma multiforme,portal cirrhosis
carnitine,1,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
GABA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
delta-aminolevulinic acid,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
leucovorin,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PGE2,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
pimecrolimus,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
auranofin,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
cefditoren,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
nitroprusside,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
mizutani = pd.read_csv("mizutani-dataset.txt", sep='\t', header=0)
mizutani # drug names are coded with pubchem ID

,abdominal.cramps,abdominal.distention,abdominal.pain,malformations,spontaneous.abortion,missed.abortion,abscess,acanthosis.nigricans,acidosis,renal.tubular.acidosis,...,vitamin.deficiency,drug.dependence,diverticulosis,prostatic.hypertrophy,allergic.reaction,dysphonia,eosinophilic.pneumonia,retinal.vein.thrombosis,renal.insufficiency,glioblastoma.multiforme
85,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
119,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
137,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
143,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
158,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6398525,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6398970,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
6447131,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
6918453,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
liu = pd.read_csv("liu-dataset.csv", sep=',', header=None)
liu

,0,1,2,3,4,5,6,7,8,9,...,1375,1376,1377,1378,1379,1380,1381,1382,1383,1384
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
827,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
828,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
829,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
830,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [6]:
# Check number of missing values
new_pauwel = pauwel.fillna(999).values
num_missing_pauwel = np.argwhere(new_pauwel == 999)
print("The number of missing values in pauwel's dataset:", len(num_missing_pauwel))

The number of missing values in pauwel's dataset: 0


In [7]:
new_mizutani = mizutani.fillna(999).values
num_missing_mizutani = np.argwhere(new_mizutani == 999)
print("The number of missing values in mizutani's dataset:",len(num_missing_mizutani))

new_liu = liu.fillna(999).values
num_missing_liu = np.argwhere(new_liu == 999)
print("The number of missing values in pauwel's dataset:" ,len(num_missing_liu))


The number of missing values in mizutani's dataset: 0
The number of missing values in pauwel's dataset: 0


In [8]:
def density(df):
    """Calculate the density of the matrix."""
    
    density = float(len(np.nonzero(df.to_numpy())[0]))
    known_se = density
    density /= (df.shape[0]*df.shape[1])
    density *= 100
    return '{:.2f}%'.format(density), known_se

In [9]:
print("Pauwel:", density(pauwel))
print("Liu:", density(liu))
print("Mizutani:", density(mizutani))

Pauwel: ('4.97%', 61102.0)
Liu: ('5.14%', 59205.0)
Mizutani: ('5.57%', 49051.0)


# Model Implementation

### Non-Numba Implementation

In [10]:
class SGD_Recommender:
    
    def __init__(self, k:int, lmbda:float, max_iter:int=350, learn_rate=0.005, tolerance=1e-06):
        """Sets the parameters for SGD."""
        
        self.k=k
        self.lmbda=lmbda
        self.max_iter=max_iter
        self.learn_rate=learn_rate
        self.tolerance=tolerance

   
            
    def fit(self, train: np.ndarray) -> None:
        """Train the SGD model.
        
        Args:            
            train (np.ndarray): The training set
        Returns: 
            None        
        """     
        np.random.seed(5)
        m, n = train.shape
        
        # Initialize the low rank matrices U and V with values from the normal distribution N(0,0.01)
        mu, sigma = 0, 0.1
        self.U = np.random.normal(loc=mu, scale=sigma, size=(m, self.k))
        self.V = np.random.normal(loc=mu, scale=sigma, size=(n, self.k))
        
        # Get non-zero values in train set
        drug, se = train.nonzero()
        drug_se = list(zip(drug, se))
        
        # Start of training
        for _ in range(self.max_iter):
            np.random.shuffle(drug_se) # Shuffle in place
            U_old = self.U.copy()
            V_old = self.V.copy()
            
            for (drug, se) in drug_se:
                error = train[drug,se] - self.predictions(self.U[drug,:], self.V[se,:])
                temp_u = self.U[drug,:] + self.learn_rate*(error*self.V[se,:] - self.lmbda*self.U[drug,:])
                temp_v = self.V[se,:] + self.learn_rate*(error*self.U[drug,:] - self.lmbda*self.V[se,:])
                self.U[drug,:] = temp_u 
                self.V[se,:] = temp_v
            
            if self.converged(U_old, self.U) and self.converged(V_old, self.V):
                break
    
    def predict(self) -> np.ndarray:
        """Predict the entire drug-side effect matrix values."""
        
        return self.predictions(self.U, self.V)

            
    def predictions(self, U: np.ndarray, V: np.ndarray) -> np.ndarray:
        """Return dot product of the matrices U and V."""
        
        return np.dot(U, V.T)
    
    def converged(self, old: np.ndarray, curr: np.ndarray) -> bool:
        """Check if matrices have reached convergence."""
        
        return np.all(np.abs(np.subtract(old,curr)) <= self.tolerance)
            
    

## Numba Implementation

In [11]:
  
@njit()
def predictions(U: np.ndarray, V: np.ndarray) -> np.ndarray:
    """Return dot product of the matrices U and V."""
    return np.dot(U, V.T)

@njit()
def converged(old: np.ndarray, curr: np.ndarray, tolerance:float=1e-06) -> bool:
    """Check if matrices have reached convergence."""

    return np.all(np.abs(np.subtract(old,curr)) <= tolerance)
    
@njit()
def fit(train: np.ndarray, k: int, lmbda: float, max_iter:int=350, learn_rate:float=0.005):
    """Train the SGD model.

    Args:
        train (np.ndarray): The training set
        k (float): Number of latent features
        lmbda (float): Regularization term.
        max_iter (int): Max number of iterations
        learn_rate (float): Learning rate
    Returns: 
        U, V (np.ndarray): The low rank matrix representation of the drug-side effect matrix.        
    """     
    np.random.seed(5)
    m, n = train.shape
    # Initialize the low rank matrices U and V with values from the normal distribution N(0,0.01)
    mu, sigma = 0, 0.1
    U = np.random.normal(loc=mu, scale=sigma, size=(m, k))
    V = np.random.normal(loc=mu, scale=sigma, size=(n, k))
    
    drug, se = train.nonzero()
    drug_se = np.array(list(zip(drug, se)))

    # Start of training
    for _ in range(max_iter):

        np.random.shuffle(drug_se) # Shuffle in place
        U_old = U.copy()
        V_old = V.copy()

        # Learn from the known  associations in the training set (drug_se)
        for (drug, se) in drug_se:
            error = train[drug,se] - predictions(U[drug,:], V[se,:])
            temp_u = U[drug,:] + learn_rate*(error*V[se,:] - lmbda*U[drug,:])
            temp_v = V[se,:] + learn_rate*(error*U[drug,:] - lmbda*V[se,:])
            U[drug,:] = temp_u 
            V[se,:] = temp_v

        if converged(U_old, U) and converged(V_old, V):
            break
    
    return U, V

@njit()    
def predict(U:np.ndarray, V:np.ndarray) -> np.ndarray:
        """Predict the entire drug-side effect matrix values."""
        
        return predictions(U, V)

# Evaluation Functions

## AUPR

In [12]:

def aupr(truth: np.ndarray, predictions: np.ndarray) -> float:
    """Get the area under the precision-recall curve, using trapezoidal rule.
    
    Args:
        truth: 1-D vector of ground truth values
        predictions: 1-D vector of predictions

    Returns:
        (float): The area.
    """
    
    max_value = predictions.max()
    min_value = predictions.min()
    # Create an array of 99 representing the thresholds
    threshold = min_value + (max_value-min_value)*np.arange(1,100,1)/100
    
    tn = np.zeros((threshold.size, 1))
    tp = np.zeros((threshold.size, 1))
    fn = np.zeros((threshold.size, 1))
    fp = np.zeros((threshold.size, 1))
    
    # Calculate the tp, tn, fp, fn for every threshold.
    for i in range(threshold.size):
        tp[i,0] = np.logical_and(predictions>=threshold[i], truth==1).sum()
        tn[i,0] = np.logical_and(predictions<threshold[i], truth==0).sum()
        fp[i,0] = np.logical_and(predictions>=threshold[i], truth==0).sum()
        fn[i,0] = np.logical_and(predictions<threshold[i], truth==1).sum()
    
    # Calculate the area under the precision-recall curve
    recall = tp/(tp+fn)
    prec = tp/(tp+fp)
    
    x = recall
    y = prec
    
    sorted_index = x.argsort(axis=0)
    y = np.take_along_axis(y, sorted_index, axis=0)
    x = np.take_along_axis(x, sorted_index, axis=0)    

    x[0][0] = 0
    y[0][0] = 1
    x = np.append(x, [[1]], axis=0)
    y = np.append(y, [[0]], axis=0)

    
    # Calculate the area using the trapezoidal rule: (b-a)*0.5*(f(b)+f(a))
    area = 0
    area = 0.5*x[0][0]*(1+y[0][0]) # still 0
    for i in range(threshold.size):
        area += (y[i][0]+ y[i+1][0])*(x[i+1][0]-x[i][0])*0.5
    return area
    

## AUROC

In [13]:

def auroc(truth: np.ndarray, predictions: np.ndarray) -> float:
    """Get the area under the ROC curve.
    
    Args:
        truth: 1-D vector of ground truth values
        predictions: 1-D vector of predictions

    Returns:
        (float): The area.
    """
    
    max_value = predictions.max()
    min_value = predictions.min()
    # Create an array of 99 representing the thresholds
    threshold = min_value + (max_value-min_value)*np.arange(1,100,1)/100
    
    tn = np.zeros((threshold.size, 1))
    tp = np.zeros((threshold.size, 1))
    fn = np.zeros((threshold.size, 1))
    fp = np.zeros((threshold.size, 1))
    
    # Calculate the tp, tn, fp, fn for every threshold.
    for i in range(threshold.size):
        tp[i,0] = np.logical_and(predictions>=threshold[i], truth==1).sum()
        tn[i,0] = np.logical_and(predictions<threshold[i], truth==0).sum()
        fp[i,0] = np.logical_and(predictions>=threshold[i], truth==0).sum()
        fn[i,0] = np.logical_and(predictions<threshold[i], truth==1).sum()
    
    # Calculate the area under the precision-recall curve
    sn = tp/(tp+fn)
    sp = tn/(tn+fp)
    x = 1 - sp
    y = sn

    sorted_index = x.argsort(axis=0)
    y = np.take_along_axis(y, sorted_index, axis=0)
    x = np.take_along_axis(x, sorted_index, axis=0)

    sorted_index = y.argsort(axis=0)
    y = np.take_along_axis(y, sorted_index, axis=0)
    x = np.take_along_axis(x, sorted_index, axis=0)

    x = np.append(x, [[1]], 0)
    y = np.append(y, [[1]], 0)
    
    # Calculate the area using the trapezoidal rule: (b-a)*0.5*(f(b)+f(a))
    area = 0
    area = 0.5*x[0][0]*y[0][0] # still 0
    for i in range(threshold.size):
        area += (y[i][0]+ y[i+1][0])*(x[i+1][0]-x[i][0])*0.5
    
    return area

## Other Metrics
Sensitivity, specificity, precision, recall, accuracy, f1-measure

In [14]:

def classification_metric(truth: np.ndarray, predictions: np.ndarray) -> tuple:
    """Calculate the evaulation metrics given 1-D vector of ground truth and predictions.
    
    Args:
        truth: 1-D vector of ground truth values
        predictions: 1-D vector of predictions

    Returns:
        sensitivity, specificity, precision, accuracy, f1
    """
    
    tp = np.logical_and(predictions==1, truth==1).sum()
    tn = np.logical_and(predictions==0, truth==0).sum()
    fp = np.logical_and(predictions==1, truth==0).sum()
    fn = np.logical_and(predictions==0, truth==1).sum()
    
    acc = (tp+tn)/(tn+tp+fn+fp)
    sn = tp/(tp+fn)
    recall = sn
    sp = tn/(tn+fp)
    prec = tp/(tp+fp)
    f1 = (2.0*prec*recall)/(recall+prec)  
      
    return sn,sp,prec,acc,f1
    

In [15]:

def get_metric(truth: np.ndarray, predictions: np.ndarray) -> tuple:
    """Calculate the metrics of the drug-side effect matrix.
    Args:
        truth: 1-D vector of ground truth values
        predictions: 1-D vector of predictions

    Returns:
        sensitivity, specificity, precision, accuracy, f1
    """

    max_value = predictions.max()
    min_value = predictions.min()
    # Create an 1-D array of 999 threshold values in ascending order
    threshold = min_value + (max_value-min_value)*np.arange(1,1000,1)/1000
    temp_sn = np.zeros(threshold.size)
    temp_sp = np.zeros(threshold.size)
    temp_prec = np.zeros(threshold.size)
    temp_acc = np.zeros(threshold.size)
    temp_f1 = np.zeros(threshold.size)

    for i in range(threshold.size):
        # assign values above threshold to 1
        predict_label = predictions>threshold[i]
        # calculate the metrics for the predictions under threshold i
        temp_sn[i],temp_sp[i],temp_prec[i],temp_acc[i],temp_f1[i] = classification_metric(truth, predict_label)
    
    # Get index corresponding to max f1 score (optimal value of prec and recall)
    indx_max_f1 = np.nanargmax(temp_f1)
    sn = temp_sn[indx_max_f1]
    sp = temp_sp[indx_max_f1]
    prec = temp_prec[indx_max_f1]
    acc = temp_acc[indx_max_f1]
    f1 = temp_f1[indx_max_f1]
    
    return sn, sp, prec, acc, f1    

# Cross-validation

In [16]:
def cross_val(cv:int, dataset:np.ndarray, lmda:float, k:int) -> np.ndarray:
    interaction_matrix = dataset
    row, col = dataset.shape
    np.random.seed(5)
    cv_matrix = np.ceil(np.random.rand(row,col)*cv)

    predict_score_matrix = np.zeros((row,col))

    for fold in range(cv):
        print("Performing CV fold: {0}".format(fold))
        test_index_matrix = (cv_matrix==fold)
        train_index_matrix = np.logical_not(test_index_matrix)
        train_interaction_matrix = np.multiply(interaction_matrix,train_index_matrix)

        # Uncomment for the non-numba implementation
        # sgd = SGD_Recommender(k=k, lmbda=lmda)
        # sgd.fit(train=train_interaction_matrix)
        # predict_matrix = sgd.predict()

        # Numba implementation
        U, V = fit(train=train_interaction_matrix, k=k, lmbda=lmda)
        predict_matrix = predict(U, V)

        predict_score_matrix = predict_score_matrix + np.multiply(predict_matrix, test_index_matrix)
    
    auc = auroc(truth=interaction_matrix.flatten(), predictions=predict_score_matrix.flatten())
    auprc = aupr(truth=interaction_matrix.flatten(), predictions=predict_score_matrix.flatten())
    sn, sp, prec, acc, f1 = get_metric(truth=interaction_matrix.flatten(), predictions=predict_score_matrix.flatten())

    return np.array([auprc, auc, sn, sp, prec, acc, f1])
        
        



# Training

## Finding the optimal parameters (currently used)
Optimal parameters are those which yield highest AUPR

In [17]:

def param_selection(dataset: np.ndarray, k_values: np.ndarray, lmda_values: np.ndarray) -> dict:
    """Calculate the metrics of the model, for each pair of given k and lambda parameter values.
    
    Args:
        dataset (np.ndarray): The dataset
        k_values (np.ndarray): List of k values
        lmda_values (np.ndarray): List of lambda values
    Returns:
        dict: Dictionary of parameter combinations and their associated score.
    """
    results = {}
    # Do 5 fold CV for each possible combination of lambda and k (product function gives the cartesian product)
    for (k,lmbda) in product(k_values, lmda_values):
        print("Trying parameters: (" , k, ", ", lmbda, ")")
        res = cross_val(cv=5, dataset=dataset, k=k, lmda=lmbda)
        print(res)
        results[(k,lmbda)] = res
    
    return results
        
    

## Training with the optimal parameters (currently used)
20 independent runs of 5 fold CV, using optimal paramters.

In [18]:
def train_optimal_params(dataset: np.ndarray, lmbda: float, k: int) -> dict:
    """Calculate the metrics of the model given k and lambda.
   
    Args:
        dataset (np.ndarray): The dataset
        k (float): k
        lmbda (float): lambda
    Returns:
        np.ndarray: array of resulting metrics.
    """
    # stores the aupr, auroc, sn, sp, prec, acc, f1
    results = np.zeros(7)
    for _ in range(20):
        res =  cross_val(cv=5, dataset=dataset, lmbda=lmbda, k=k)
        # element wise addition
        results = results + res    
    
    # return the mean over the 2 independent runs
    return results/20.0

## Training Pauwel's Dataset

Possible k values = 1,5,10,15,20,25,30,35,40,50,100

Possible lambda values = 0.1,1,5,10,15,20

In [19]:
%time param_selection(dataset=pauwel.values.copy(), k_values=np.array([1,5,10,15,20,25,30,35,40,50,100]), lmda_values=np.array([0.1,1,5,10,15,20]))

Trying parameters: ( 1 ,  0.1 )
Performing CV fold: 0
Performing CV fold: 1
Performing CV fold: 2
Performing CV fold: 3
Performing CV fold: 4


/tmp/ipykernel_4395/1932564076.py:22: RuntimeWarning: invalid value encountered in double_scalars
  f1 = (2.0*prec*recall)/(recall+prec)


[0.15331458 0.62777279 0.32435927 0.90349408 0.1494499  0.87472192
 0.20462019]
Trying parameters: ( 1 ,  1.0 )
Performing CV fold: 0
Performing CV fold: 1
Performing CV fold: 2
Performing CV fold: 3
Performing CV fold: 4
[0.02485349 0.500254   0.99854342 0.01469056 0.05031498 0.06356962
 0.09580262]
Trying parameters: ( 1 ,  5.0 )
Performing CV fold: 0
Performing CV fold: 1
Performing CV fold: 2
Performing CV fold: 3
Performing CV fold: 4
[2.48407156e-02 5.00000856e-01 1.00000000e+00 1.36895116e-05
 4.96819160e-02 4.96942791e-02 9.46608973e-02]
Trying parameters: ( 1 ,  10.0 )
Performing CV fold: 0
Performing CV fold: 1
Performing CV fold: 2
Performing CV fold: 3
Performing CV fold: 4
[2.48407156e-02 5.00000856e-01 1.00000000e+00 1.36895116e-05
 4.96819160e-02 4.96942791e-02 9.46608973e-02]
Trying parameters: ( 1 ,  15.0 )
Performing CV fold: 0
Performing CV fold: 1
Performing CV fold: 2
Performing CV fold: 3
Performing CV fold: 4
[2.48407156e-02 5.00000856e-01 1.00000000e+00 1.368951

{(1,
  0.1): array([0.15331458, 0.62777279, 0.32435927, 0.90349408, 0.1494499 ,
        0.87472192, 0.20462019]),
 (1,
  1.0): array([0.02485349, 0.500254  , 0.99854342, 0.01469056, 0.05031498,
        0.06356962, 0.09580262]),
 (1,
  5.0): array([2.48407156e-02, 5.00000856e-01, 1.00000000e+00, 1.36895116e-05,
        4.96819160e-02, 4.96942791e-02, 9.46608973e-02]),
 (1,
  10.0): array([2.48407156e-02, 5.00000856e-01, 1.00000000e+00, 1.36895116e-05,
        4.96819160e-02, 4.96942791e-02, 9.46608973e-02]),
 (1,
  15.0): array([2.48407156e-02, 5.00000856e-01, 1.00000000e+00, 1.36895116e-05,
        4.96819160e-02, 4.96942791e-02, 9.46608973e-02]),
 (1,
  20.0): array([2.48407156e-02, 5.00000856e-01, 1.00000000e+00, 1.36895116e-05,
        4.96819160e-02, 4.96942791e-02, 9.46608973e-02]),
 (5,
  0.1): array([0.09181122, 0.63345994, 0.49360086, 0.86170085, 0.15724631,
        0.84341318, 0.23851042]),
 (5,
  1.0): array([0.02777163, 0.49777602, 0.79113613, 0.32194309, 0.05749027,
       

In [ ]:
print("Now training with optimal params.....")
lmda=0.1
k=1
# 20 runs of 5-CV with optimal parameters
%time train_optimal_params(dataset=pauwel.values.copy(), lmbda=lmda, k=k)


In [ ]:
# Finding optimal parameters using Mizutani's Dataset
%time param_selection(dataset=mizutani.values.copy(), k_values=np.array([1,5,10,15,20,25,30,35,40,50,100]), lmda_values=np.array([0.1,1,5,10,15,20]))

In [ ]:
print("Now training with optimal params.....")
lmda=0.1
k=1
# 20 runs of 5-CV with optimal parameters
%time train_optimal_params(dataset=mizutani.values.copy(), lmbda=lmda, k=k)


In [ ]:
%time param_selection(dataset=liu.values.copy(), k_values=np.array([1,5,10,15,20,25,30,35,40,50,100]), lmda_values=np.array([0.1,1,5,10,15,20]))

In [ ]:
print("Now training with optimal params.....")
lmda=0.1
k=1
# 20 runs of 5-CV with optimal parameters
%time train_optimal_params(dataset=liu.values.copy(), lmbda=lmda, k=k)
